In [1]:
%load_ext autoreload
%autoreload 2

import logging
logging.getLogger("querychains").setLevel(logging.DEBUG)
import querychains
from querychains import with_context, Context, FileStorage, start_server

In [2]:
storage = FileStorage("data")
storage

<FileStorage directory='/home/ada/projects/querychain/data'>

In [3]:
# Register context into storage

with Context("root1", storage=storage):
    pass


In [4]:
# Manual writing context into storage

with Context("root2") as c:
    pass

storage.write_context(c)

In [6]:
# Running data browser over storage

server = start_server(storage=storage)

INFO:     Started server process [120844]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:46759 (Press CTRL+C to quit)


INFO:     127.0.0.1:55326 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:55326 - "GET /static/js/main.9895fe90.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:55332 - "GET /static/css/main.073c9b0a.css HTTP/1.1" 200 OK
INFO:     127.0.0.1:55332 - "GET /contexts/list HTTP/1.1" 200 OK
INFO:     127.0.0.1:55332 - "GET /logo192.png HTTP/1.1" 200 OK
INFO:     127.0.0.1:55326 - "GET /favicon.ico HTTP/1.1" 200 OK
INFO:     127.0.0.1:55332 - "GET /contexts/uuid/c64db2c8-5343-4e64-bf97-4de6199f225f HTTP/1.1" 200 OK
INFO:     127.0.0.1:55326 - "GET /contexts/uuid/4954f653-8924-48c6-b6b7-71ca29d4922a HTTP/1.1" 200 OK
INFO:     127.0.0.1:55326 - "GET /contexts/uuid/efa7b6fd-b8f8-4465-a085-f42683520c06 HTTP/1.1" 200 OK
INFO:     127.0.0.1:55326 - "GET /contexts/uuid/cff44a31-5efd-4b3e-8098-b118b1ba9afd HTTP/1.1" 200 OK
INFO:     127.0.0.1:55326 - "GET /contexts/uuid/56df9166-a677-44af-afc2-ff8addad3ed4 HTTP/1.1" 200 OK
INFO:     127.0.0.1:55326 - "GET /contexts/uuid/e497c356-6e0c-45d8-ab6a-9ea32a98898a HTTP